## Importance of Feature Selection

In this notebook I aim to study the importance of feature selection while building a Deep Learning model. I hypothesize that a model would be able to find out the most importance features. So, we can throw all variables in the deep learning models and its performance will not be deteriorated. 

I will check and compare the performance of models built using all features.

### Importing Required Libraries and Data

In [2]:
import pandas as pd
import numpy as np

#to plot the data
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

import time
import os
os.chdir("C:/Users/palig\MSE/Vertiefungsprojekt1/MSE_Vertierfungsprojekt1/data/raw/CMaps/")

from sklearn.preprocessing import MinMaxScaler #to normalize data
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score

#for deep learning
import keras
import keras.backend as k
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Masking, Dropout
from keras.optimizers import RMSprop, Adam
from keras.callbacks import History
from keras import callbacks

In [3]:
def prepare_data(drop_cols = True):
    dependent_var = ['RUL']
    index_columns_names =  ["UnitNumber","Cycle"]
    operational_settings_columns_names = ["OpSet"+str(i) for i in range(1,4)]
    sensor_measure_columns_names =["SensorMeasure"+str(i) for i in range(1,22)]
    input_file_column_names = index_columns_names + operational_settings_columns_names + sensor_measure_columns_names

    cols_to_drop = ['OpSet3', 'SensorMeasure1', 'SensorMeasure5', 'SensorMeasure6', 'SensorMeasure10', 'SensorMeasure14',
     'SensorMeasure16', 'SensorMeasure18', 'SensorMeasure19']

    df_train = pd.read_csv('train_FD001.txt',delim_whitespace=True,names=input_file_column_names)

    rul = pd.DataFrame(df_train.groupby('UnitNumber')['Cycle'].max()).reset_index()
    rul.columns = ['UnitNumber', 'max']
    df_train = df_train.merge(rul, on=['UnitNumber'], how='left')
    df_train['RUL'] = df_train['max'] - df_train['Cycle']
    df_train.drop('max', axis=1, inplace=True)

    df_test = pd.read_csv('test_FD001.txt', delim_whitespace=True, names=input_file_column_names)
    
    if(drop_cols == True):
        df_train = df_train.drop(cols_to_drop, axis = 1)
        df_test = df_test.drop(cols_to_drop, axis = 1)

    y_true = pd.read_csv('RUL_FD001.txt', delim_whitespace=True,names=["RUL"])
    y_true["UnitNumber"] = y_true.index
    
    return df_train, df_test, y_true

In [4]:
df_train, df_test, y_true = prepare_data(drop_cols=False)
df_train.shape, df_test.shape, y_true.shape

C:\Users\palig\AppData\Local\Temp\ipykernel_11160\860272957.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_train = pd.read_csv('train_FD001.txt',delim_whitespace=True,names=input_file_column_names)
C:\Users\palig\AppData\Local\Temp\ipykernel_11160\860272957.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_test = pd.read_csv('test_FD001.txt', delim_whitespace=True, names=input_file_column_names)
C:\Users\palig\AppData\Local\Temp\ipykernel_11160\860272957.py:25: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y_true = pd.read_csv('RUL_FD001.txt', delim_whitespace=True,names=["RUL"])


((20631, 27), (13096, 26), (100, 2))

In [5]:
feats = df_train.columns.drop(['UnitNumber', 'Cycle', 'RUL'])

In [6]:
min_max_scaler = MinMaxScaler(feature_range=(-1,1))

df_train[feats] = min_max_scaler.fit_transform(df_train[feats])
df_test[feats] = min_max_scaler.transform(df_test[feats])

In [7]:
df_train['failure'] = [1 if i < 50 else 0 for i in df_train.RUL]
y_true['failure'] = [1 if i < 50 else 0 for i in y_true.RUL]

In [8]:
df_train.head()

,UnitNumber,Cycle,OpSet1,OpSet2,OpSet3,SensorMeasure1,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure5,...,SensorMeasure14,SensorMeasure15,SensorMeasure16,SensorMeasure17,SensorMeasure18,SensorMeasure19,SensorMeasure20,SensorMeasure21,RUL,failure
0,1,1,-0.080460,-0.666667,-1.0,-1.0,-0.632530,-0.186396,-0.380486,-1.0,...,-0.600784,-0.272028,-1.0,-0.333333,-1.0,-1.0,0.426357,0.449323,191,0
1,1,2,0.218391,-0.500000,-1.0,-1.0,-0.433735,-0.093961,-0.294733,-1.0,...,-0.674373,-0.177376,-1.0,-0.333333,-1.0,-1.0,0.333333,0.462027,190,0
2,1,3,-0.494253,0.500000,-1.0,-1.0,-0.313253,-0.260955,-0.258947,-1.0,...,-0.656414,-0.285110,-1.0,-0.666667,-1.0,-1.0,0.255814,0.242751,189,0
3,1,4,0.080460,0.000000,-1.0,-1.0,-0.313253,-0.487683,-0.337610,-1.0,...,-0.650222,-0.666795,-1.0,-0.333333,-1.0,-1.0,0.147287,0.324772,188,0
4,1,5,-0.218391,-0.333333,-1.0,-1.0,-0.301205,-0.485066,-0.190749,-1.0,...,-0.650532,-0.195845,-1.0,-0.166667,-1.0,-1.0,0.178295,0.409003,187,0


In [9]:
df_test.head()

,UnitNumber,Cycle,OpSet1,OpSet2,OpSet3,SensorMeasure1,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure5,...,SensorMeasure12,SensorMeasure13,SensorMeasure14,SensorMeasure15,SensorMeasure16,SensorMeasure17,SensorMeasure18,SensorMeasure19,SensorMeasure20,SensorMeasure21
0,1,1,0.264368,0.500000,-1.0,-1.0,0.090361,-0.378679,-0.461175,-1.0,...,0.292111,-0.558824,-0.735680,-0.382070,-1.0,-0.333333,-1.0,-1.0,0.116279,0.323667
1,1,2,-0.310345,-0.500000,-1.0,-1.0,-0.698795,-0.240898,-0.555368,-1.0,...,0.479744,-0.470588,-0.590463,-0.573682,-1.0,-0.166667,-1.0,-1.0,0.364341,0.373654
2,1,3,0.034483,0.166667,-1.0,-1.0,-0.246988,-0.306736,-0.355503,-1.0,...,0.398721,-0.558824,-0.688719,-0.082724,-1.0,-0.166667,-1.0,-1.0,0.457364,0.442695
3,1,4,0.482759,0.000000,-1.0,-1.0,-0.259036,-0.429693,-0.183997,-1.0,...,0.147122,-0.500000,-0.659820,-0.485956,-1.0,-0.500000,-1.0,-1.0,0.333333,0.324220
4,1,5,0.160920,0.000000,-1.0,-1.0,-0.216867,-0.295836,-0.335922,-1.0,...,0.475480,-0.558824,-0.694499,-0.398230,-1.0,-0.666667,-1.0,-1.0,0.317829,0.432753


In [10]:
y_true.head()

,RUL,UnitNumber,failure
0,112,0,0
1,98,1,0
2,69,2,0
3,82,3,0
4,91,4,0



LSTM expects an input in the shape of a numpy array of 3 dimensions and I will need to convert train and test data accordingly.

In [11]:
def gen_train(id_df, seq_length, seq_cols):
    """
        function to prepare train data into (samples, time steps, features)
        id_df = train dataframe
        seq_length = look back period
        seq_cols = feature columns
    """
        
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array=[]
    
    for start, stop in zip(range(0, num_elements-seq_length+1), range(seq_length, num_elements+1)):
        lstm_array.append(data_array[start:stop, :])
    
    return np.array(lstm_array)

In [12]:
def gen_target(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length-1:num_elements+1]

In [18]:
def gen_test(id_df, seq_length, seq_cols, mask_value):
    """
        function to prepare test data into (samples, time steps, features)
        function only returns last sequence of data for every unit
        id_df = test dataframe
        seq_length = look back period
        seq_cols = feature columns
    """
    df_mask = pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    df_mask[:] = mask_value
    
    id_df = pd.concat([df_mask, id_df], ignore_index=True)
    
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array=[]
    
    start = num_elements-seq_length
    stop = num_elements
    
    lstm_array.append(data_array[start:stop, :])
    
    return np.array(lstm_array)


Let's define look back period and mask_value

In [19]:
sequence_length = 50 #predicting using last 30 cycle values
mask_value = 0


Let's prepare data using above functions.

In [20]:
#generate train
x_train=np.concatenate(list(list(gen_train(df_train[df_train['UnitNumber']==unit], sequence_length, feats)) for unit in df_train['UnitNumber'].unique()))
print(x_train.shape)

(15731, 50, 24)


In [21]:
#generate target of train
y_train = np.concatenate(list(list(gen_target(df_train[df_train['UnitNumber']==unit], sequence_length, "failure")) for unit in df_train['UnitNumber'].unique()))
y_train.shape

(15731,)

In [22]:
#generate test
x_test=np.concatenate(list(list(gen_test(df_test[df_test['UnitNumber']==unit], sequence_length, feats, mask_value)) for unit in df_test['UnitNumber'].unique()))
print(x_test.shape)

(100, 50, 24)


In [27]:
display(x_train)

array([[[-0.08045977, -0.66666667, -1.        , ..., -1.        ,
          0.42635659,  0.44932339],
        [ 0.2183908 , -0.5       , -1.        , ..., -1.        ,
          0.33333333,  0.46202706],
        [-0.49425287,  0.5       , -1.        , ..., -1.        ,
          0.25581395,  0.24275062],
        ...,
        [ 0.2183908 ,  0.16666667, -1.        , ..., -1.        ,
          0.24031008,  0.30019332],
        [-0.16091954,  0.83333333, -1.        , ..., -1.        ,
          0.42635659,  0.53438277],
        [-0.36781609, -0.16666667, -1.        , ..., -1.        ,
          0.10077519,  0.42612538]],

       [[ 0.2183908 , -0.5       , -1.        , ..., -1.        ,
          0.33333333,  0.46202706],
        [-0.49425287,  0.5       , -1.        , ..., -1.        ,
          0.25581395,  0.24275062],
        [ 0.08045977,  0.        , -1.        , ..., -1.        ,
          0.14728682,  0.32477216],
        ...,
        [-0.16091954,  0.83333333, -1.        , ..., -

In [23]:
#true target of test 
y_test = y_true.RUL.values
y_test.shape

(100,)

In [24]:
nb_features = x_train.shape[2]
nb_out = 1

In [25]:
nb_features

24

Hier wird die Ungleichheit zwischen failure=0 und failure=1 berechnet. Dadurch entsteht eine Gewichtung, die verwendet werden sollte.

In [30]:
class_0 = pd.Series(y_train).value_counts()[0]
class_1 = pd.Series(y_train).value_counts()[1]
total = class_0 + class_1

cls_wt = {0: class_1/total, 1: class_0/total}

In [31]:
cls_wt

{0: 0.3178437480134766, 1: 0.6821562519865234}

### Experiment

Test if GPU is available

In [36]:
import tensorflow as tf

# Überprüfe, ob GPUs verfügbar sind
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [41]:
def run_exp_model(X, y, X_test, y_test, nruns=10):
    """
    Parameters:
    X, y = training pandas dataframe
    X_test, y_test = testing pandas dataframe
    nruns = number of times the models should run
    
    return:
    Dataframe with Model Run, Accuracy, Precision, Recall, F1 Score, AUC Score
    """
    
    score_df = pd.DataFrame(columns=["Model", "Accuracy", "Precision", "Recall", "F1 Score", "AUC Score"])

    for i in range(0, nruns):
        start = time.time()

        with tf.device('/GPU:0'):
            model = Sequential()
            model.add(LSTM(
                    units=8,
                    return_sequences=True,
                    input_shape=(sequence_length, nb_features)))
            model.add(Dropout(0.2))
            model.add(LSTM(
                    units=4,
                    return_sequences=False))
            model.add(Dropout(0.2))
            model.add(Dense(units=1, activation='sigmoid'))

            model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])

            model.fit(X, y, epochs=100, batch_size=64, validation_split=0.1, verbose=0, class_weight=cls_wt,
                      shuffle=True, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, 
                                                                            verbose=0, mode='auto')])

            # Ersetze predict_classes mit predict und einem Schwellenwert
            predictions = (model.predict(X_test) > 0.5).astype("int32")

            acc = accuracy_score(y_test, predictions)
            pr = precision_score(y_test, predictions)
            rc = recall_score(y_test, predictions)
            f1 = f1_score(y_test, predictions)
            auc = roc_auc_score(y_test, predictions)

            # Verwende pd.concat anstelle von append
            new_row = pd.DataFrame({'Model': [i + 1], 'Accuracy': [acc], 'Precision': [pr], 
                                    'Recall': [rc], 'F1 Score': [f1], 'AUC Score': [auc]})
            score_df = pd.concat([score_df, new_row], ignore_index=True)

            end = time.time()
            print(f"Run: {i+1} and Runtime: {np.round(end - start, 3)} Seconds")        
    
    return score_df


In [42]:
result_df = run_exp_model(x_train, y_train, x_test, y_true.failure, nruns = 10)

4/4 [==============================] - 0s 3ms/step
Run: 1 and Runtime: 33.605 Seconds


C:\Users\palig\AppData\Local\Temp\ipykernel_11160\2109221406.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  score_df = pd.concat([score_df, new_row], ignore_index=True)


4/4 [==============================] - 0s 3ms/step
Run: 2 and Runtime: 29.121 Seconds
4/4 [==============================] - 0s 4ms/step
Run: 3 and Runtime: 32.909 Seconds
4/4 [==============================] - 0s 3ms/step
Run: 4 and Runtime: 49.919 Seconds
4/4 [==============================] - 0s 3ms/step
Run: 5 and Runtime: 32.697 Seconds
4/4 [==============================] - 0s 3ms/step
Run: 6 and Runtime: 53.46 Seconds
4/4 [==============================] - 0s 3ms/step
Run: 7 and Runtime: 51.076 Seconds
4/4 [==============================] - 0s 3ms/step
Run: 8 and Runtime: 31.891 Seconds
4/4 [==============================] - 0s 3ms/step
Run: 9 and Runtime: 36.539 Seconds
4/4 [==============================] - 0s 3ms/step
Run: 10 and Runtime: 45.258 Seconds


In [43]:
result_df

,Model,Accuracy,Precision,Recall,F1 Score,AUC Score
0,1,0.96,0.882353,1.0,0.937500,0.971429
1,2,0.95,0.857143,1.0,0.923077,0.964286
2,3,0.97,0.909091,1.0,0.952381,0.978571
3,4,0.94,0.833333,1.0,0.909091,0.957143
4,5,0.97,0.909091,1.0,0.952381,0.978571
5,6,0.96,0.882353,1.0,0.937500,0.971429
6,7,0.95,0.857143,1.0,0.923077,0.964286
7,8,0.97,0.909091,1.0,0.952381,0.978571
8,9,0.96,0.882353,1.0,0.937500,0.971429
9,10,0.95,0.857143,1.0,0.923077,0.964286


In [45]:
fig, ax = plt.subplots(figsize = (8,6))
result_df.drop(['Model'], axis = 1).boxplot(vert = 1, ax = ax)
ax.set_title('Binary Claasification Model Performance on Test Data : All features')
ax.grid(visible=False)
ax.set_ylabel('Value')

Text(0, 0.5, 'Value')

In [30]:
result_df.mean()

Model        5.500000
Accuracy     0.965000
Precision    0.901916
Recall       0.993333
AUC Score    0.973095
F1 Score     0.944916
dtype: float64